In [1]:
from qumran_seagulls.types import *
from qumran_seagulls.utils import *
from matplotlib import pyplot as plt

In [2]:
path = 'data/lines_cropped/P123-Fg001-R-C01-R01'

In [3]:
import torch

In [4]:
os.listdir(path)

['line_8.jpg',
 'line_7.jpg',
 'line_2.jpg',
 'line_5.jpg',
 'line_9.jpg',
 'line_11.jpg',
 'line_6.jpg',
 'data.csv',
 'line_1.jpg',
 'line_10.jpg',
 'line_3.jpg',
 'line_12.jpg',
 'line_4.jpg']

In [5]:
lines = [cv2.imread(os.path.join(path, f), 0) for f in os.listdir(path) if f.endswith('jpg')]
len(lines)
[l.shape for l in lines]

[(214, 1870),
 (236, 2014),
 (171, 1644),
 (196, 2324),
 (213, 1754),
 (164, 1518),
 (208, 2156),
 (253, 659),
 (236, 1582),
 (219, 2415),
 (184, 1406),
 (197, 2027)]

In [6]:
show(lines[1])
lines = [remove_blobs(thresh_invert(line), 10) for line in lines]
show(lines[0])

In [107]:

def histogram_cleaning(line: array, width_thresh: int = 15, span_thresh: int = 30) -> List[array]:
    histogram = np.where(line > 0, 1, 0).sum(axis=0)
    zeros = np.where(histogram==0)[0]
    change_idces = np.where(np.diff(zeros) > 1)[0]
    starts = [zeros[i] for i in change_idces] 
    ends = [zeros[i+1] for i in change_idces]

    # remove very narrow zero areas as they might be of the same character
    widths = [e - s for s, e in zip(starts, ends)]
    remove = [i for i, w in enumerate(widths) if w < width_thresh]
    ends = [e for i, e in enumerate(ends) if i not in remove]
    starts = [s for i, s in enumerate(starts) if i not in [j+1 for j in remove]]

    # remove very narrow spans for the same reason
    spans = [starts[i+1] - e for i, e in enumerate(ends[:-1])]
    remove = [i for i, s in enumerate(spans) if s < span_thresh]
    ends = [e for i, e in enumerate(ends) if i not in remove]
    starts = [s for i, s in enumerate(starts) if i not in [j+1 for j in remove]]
    #visualize(line, histogram, starts, ends)

    return [line[:, s:e+16] for s, e in zip(starts, ends)]


In [108]:
#start with a line
line = lines[3]

In [109]:
def create_windows(image: array, win_width: int = 50, win_step: int = 10) -> List[array]:
    num_windows = ceil((image.shape[1] - win_width) / win_step)
    windows = [image[:, i*win_step : win_width + i*win_step] for i in range(num_windows)]
    return [w for w in windows if w.min() == 0]

    


In [110]:
from qumran_seagulls.preprocess.char_segm.char_segm import histogram_cleaning

crops = histogram_cleaning(line)
all_windows = [create_windows(w) for w in crops]
crops = [cs for i, cs in enumerate(crops) if len(all_windows[i]) > 0]
all_windows = [ws for ws in all_windows if len(ws) > 0]


In [111]:
len(crops), len(all_windows)

(11, 11)

In [112]:
from qumran_seagulls.models.cnn import default_cnn_monkbrill
model = default_cnn_monkbrill().eval()
model.load_pretrained('checkpoints/cnn_labels_augm_fuzzy.p')

In [113]:
all_scores = [model.predict_scores(wins) for wins in all_windows]

In [114]:
    def get_likelihoods(crop: array, scores: Tensor) -> Tensor:
        # heuristic
        num_chars = crop.shape[1] // 64 + 1
        max_probs = [score.softmax(-1).max() for score in scores]
        minima = find_peaks(-array(max_probs), prominence=0.01)[0]
        
        minima = [m for m in minima if m not in [1, len(max_probs)-2]]
        if len(minima) > num_chars-1:
            values = [max_probs[i] for i in minima]
            minima = [minima[i] for i, p  in enumerate(values) if p in sorted(values)[:num_chars-1]]
        
        if num_chars == 1 or not len(minima):
            lkhds = [scores.mean(0).softmax(-1)]

        else:
            # average over sliding windows for each char range
            lkhds = [scores[:minima[0]].mean(0)]
            lkhds.extend([scores[m+1: minima[i+1]].mean(0) for i, m in enumerate(minima[:-1])])
            lkhds.append(scores[minima[-1] + 1:].mean(0))
            lkhds = [l.softmax(-1) for l in lkhds if not torch.isnan(l[0])]

        return torch.stack(lkhds)

In [144]:
ind = 11
crop, scores = crops[ind], all_scores[ind]
from scipy.signal import find_peaks
show(crop)

num_chars = crop.shape[1] // 64 + 1
max_probs = [score.softmax(-1).max().item() for score in scores]
minima = find_peaks(-np.array(max_probs), prominence=0.01)[0]
maxima = find_peaks(np.array(max_probs))[0]
print(crop.shape, num_chars)

print('num windows', len(max_probs))
print('num minima before', len(minima), minima)
minima = [m for m in minima if m not in [1, len(max_probs)-2]]
if len(minima) > num_chars-1:
    values = [max_probs[i] for i in minima]
    minima = [minima[i] for i, p  in enumerate(values) if p in sorted(values)[:num_chars-1]]
    
print(minima)
print(max_probs)
plt.plot(max_probs)
plt.vlines(minima, 0, 1, color="C1")
plt.vlines(maxima, 0, 1, color="C3", linestyles='dotted')
plt.grid(True)
plt.show()

Ls = get_likelihoods(crop, scores)
print(Ls.shape)


IndexError: list index out of range

In [142]:
path, qs, Qfinal = Viterbi(torch.flip(Ls, [0]))
print(path)
print([LABEL_MAP[s.argmax(-1).item()] for s in scores])
print([LABEL_MAP[i] for i in path])

[3, 24, 2]
['Dalet', 'Resh', 'Tsadi-final', 'Het', 'Het', 'Waw', 'Zayin', 'Lamed', 'Tet', 'Bet', 'Bet', 'Bet']
['Dalet', 'Waw', 'Bet']


In [19]:
from qumran_seagulls.models.viterbi import default_viterbi
Viterbi = default_viterbi('cpu')

In [60]:
class Viterbi(object):
    def __init__(self, 
                 num_classes: int, 
                 transition_matrix: Tensor,
                 T_sos: Tensor,
                 T_eos: Tensor):
        super().__init__()
        self.num_classes = num_classes
        self.transition = transition_matrix      # K x K
        self.T_sos = T_sos 
        self.T_eos = T_eos        

    @torch.no_grad()
    def __call__(self, likelihoods: Tensor) -> Tuple[List[int], Tensor, float]:
        T = len(likelihoods) # sequence length

        if T == 1:
            # return only from input transition
            _q = likelihoods[0, :] * self.T_sos
            return [_q.argmax().item()], _q, _q.max().item()

        K = self.num_classes

        qs = torch.empty((T, K), device=likelihoods.device)         
        paths = torch.empty((T-1, K), dtype=int, device=qs.device)
        qs[0, :] = likelihoods[0, :] * self.T_sos 
        for step in range(1, T):
            # @todo: find the broadcasting magic
            for j in range(K):
                query = (qs[step-1,:] * self.transition.T[j] * likelihoods[step, j])
                paths[step-1, j] = query.argmax()
                qs[step, j] = query.max()
        last_path = (qs[step, :] * self.T_eos).argmax().item()
        qs_end = (qs[step, :] * self.T_eos).max().item()

        best_path = [last_path]
        for step in range(T-1):
            last_path = paths[-1 - step, last_path].item()
            best_path.append(last_path)
            
        return best_path[::-1], qs, qs_end


def default_viterbi(device: str):
    # remove transition probs from corresponding medial and final characters
    T_sos = torch.tensor([1/23 if i not in [8, 12, 15, 22] else 0 for i in range(27)], device=device)
    T_eos = torch.tensor([1/24 if i not in [11, 13, 23] else 0 for i in range(27)], device=device)
    
    return Viterbi(num_classes=27, transition_matrix=torch.load('checkpoints/trans.p'), T_sos = T_sos, T_eos = T_eos)

Viterbi = default_viterbi('cpu')

In [51]:
path, qs, Qfinal = Viterbi(torch.flip(Ls, [0]))

print([LABEL_MAP[i] for i in path])

['Alef', 'Alef', 'Alef', 'Kaf-final', 'Resh']


In [28]:
num_chars = crop.shape[1] // 64 + 1
max_probs = [score.softmax(-1).max().item() for score in scores]
minima = find_peaks(-np.array(max_probs), prominence=0.01)[0]
maxima = find_peaks(np.array(max_probs))[0]
minima = [m for m in minima if m not in [1, len(max_probs)-2]]
if len(minima) > num_chars-1:
    values = [max_probs[i] for i in minima]
    minima = [minima[i] for i, p  in enumerate(values) if p in sorted(values)[:num_chars-1]]
    
lhds = [scores[:minima[0]].mean(0)]
lhds += [scores[m+1: minima[i+1]].mean(0) for i, m in enumerate(minima[:-1])]
lhds.append(scores[minima[-1] + 1:].mean(0))
print(len(lhds))

5
